# Segment Surface of Hippocampal Formation

This notebook transforms the 3D segmentation images into 3D meshed surfaces and stores them into .ply files.

The meshes in the .ply files can be opened with:
- the `vscode-3d-preview` extension of VSCode
- [MeshLab](https://www.meshlab.net/)
- [Blender](https://www.blender.org/download/), specifically with its [Stop-motion-OBJ](https://github.com/neverhood311/Stop-motion-OBJ) plugin to visualize time-series of .ply files.

**Note:** Add a plugin to blender following [these instructions](https://docs.blender.org/manual/en/latest/editors/preferences/addons.html#rd-party-add-ons). 

## Set-up

In [1]:
import os
import subprocess

gitroot_path = subprocess.check_output(
    ["git", "rev-parse", "--show-toplevel"], universal_newlines=True
)
os.chdir(gitroot_path[:-1])
print("Working directory: ", os.getcwd())

import warnings

warnings.filterwarnings("ignore")

import sys

sys_dir = os.path.dirname(os.getcwd())
sys.path.append(sys_dir)
print("Directory added to path: ", sys_dir)

Working directory:  /home/nmiolane/code/my28brains
Directory added to path:  /home/nmiolane/code


## Imports

In [6]:
import glob

import nibabel
import numpy as np
import skimage
import trimesh

import my28brains.io as io

IMG_DIR = "/home/data/28andme/"
MESHES_DIR = os.path.join(os.getcwd(), "data", "meshes")

print("IMG_DIR: ", IMG_DIR)
print("MESHES_DIR: ", MESHES_DIR)

IMG_DIR:  /home/data/28andme/
MESHES_DIR:  /home/nmiolane/code/my28brains/data/meshes


## Functions to Extract Meshes

In [3]:
def _extract_mesh(img_fdata, structure_id):
    """Extract one surface mesh from the fdata of a segmented image."""
    if structure_id == -1:
        img_mask = img_fdata != 0
    else:
        img_mask = img_fdata == structure_id
    meshing_result = skimage.measure.marching_cubes(img_mask)
    mesh = trimesh.Trimesh(vertices=meshing_result[0], faces=meshing_result[1])
    return mesh


def extract_mesh(nii_path, structure_id=-1):
    """Extract surface mesh(es) from a structure in the hippocampal formation.

    The nii_path should contain the segmentation of the hippocampal formation,
    with the following structures (given from their corresponding integer labels):
    1: CA1
    2: CA23
    3: Dentate Gyrus (DG)
    4: Entorhinal Cortex (ERC)
    5: Perirhinal Cortex (PRC)
    6: Parahippocampal Cortex (PHC)
    7: Subiculum (SUB)
    8: Anterior Hipp
    9: Posterior Hipp

    Note that the label 0 denotes the background.

    Parameters
    ----------
    nii_path : str
        Path of the .nii.gz image with the segmented structures.
    structure_id : int or list
        Index or list of indices of the anatomical structure(s) to
        mesh from the hippocampal formation.
        Possible indices are either -1 (entire structure) or any of the
        labels of the segmentation.

    Return
    ------
    mesh : trimesh.Trimesh or list of trimesh.Trimesh's
        Surface mesh (or list of meshes) of the anatomical structure.
    """
    print(f"Loading image from {nii_path}...")
    img = nibabel.load(nii_path)
    img_fdata = img.get_fdata()

    if isinstance(structure_id, int):
        return _extract_mesh(img_fdata, structure_id)

    meshes = []
    for one_structure_id in structure_id:
        one_mesh = _extract_mesh(img_fdata, one_structure_id)
        meshes.append(one_mesh)
    return meshes

## Left Hemisphere: Extract and Write Meshes

In [4]:
hemisphere = "left"
structure_id = -1

string_base = os.path.join(
    IMG_DIR, f"sub-01/ses-**/ashs/{hemisphere}_lfseg_corr_usegray_CT_LQ.nii.gz"
)
nii_paths = sorted(glob.glob(string_base))

print(f"Found {len(nii_paths)} image paths:\n {nii_paths}\n")

for nii_path in nii_paths:
    mesh = extract_mesh(nii_path=nii_path, structure_id=structure_id)

    sub, ses = nii_path.split("/")[4:6]
    ply_path = os.path.join(
        MESHES_DIR, f"{hemisphere}_structure_{structure_id}_{sub}_{ses}.ply"
    )
    io.write_trimesh_to_ply(mesh=mesh, ply_path=ply_path)

Found 27 image paths:
 ['/home/data/28andme/sub-01/ses-02/ashs/left_lfseg_corr_usegray_CT_LQ.nii.gz', '/home/data/28andme/sub-01/ses-03/ashs/left_lfseg_corr_usegray_CT_LQ.nii.gz', '/home/data/28andme/sub-01/ses-04/ashs/left_lfseg_corr_usegray_CT_LQ.nii.gz', '/home/data/28andme/sub-01/ses-05/ashs/left_lfseg_corr_usegray_CT_LQ.nii.gz', '/home/data/28andme/sub-01/ses-06/ashs/left_lfseg_corr_usegray_CT_LQ.nii.gz', '/home/data/28andme/sub-01/ses-07/ashs/left_lfseg_corr_usegray_CT_LQ.nii.gz', '/home/data/28andme/sub-01/ses-08/ashs/left_lfseg_corr_usegray_CT_LQ.nii.gz', '/home/data/28andme/sub-01/ses-09/ashs/left_lfseg_corr_usegray_CT_LQ.nii.gz', '/home/data/28andme/sub-01/ses-10/ashs/left_lfseg_corr_usegray_CT_LQ.nii.gz', '/home/data/28andme/sub-01/ses-11/ashs/left_lfseg_corr_usegray_CT_LQ.nii.gz', '/home/data/28andme/sub-01/ses-12/ashs/left_lfseg_corr_usegray_CT_LQ.nii.gz', '/home/data/28andme/sub-01/ses-14/ashs/left_lfseg_corr_usegray_CT_LQ.nii.gz', '/home/data/28andme/sub-01/ses-15/ashs/l

## Right Hemisphere: Segment and Write Meshes

In [5]:
hemisphere = "right"
structure_id = -1

string_base = os.path.join(
    IMG_DIR, f"sub-01/ses-**/ashs/{hemisphere}_lfseg_corr_usegray_CT_LQ.nii.gz"
)
nii_paths = sorted(glob.glob(string_base))
print(f"Found {len(nii_paths)} image paths:\n {nii_paths}\n")

for nii_path in nii_paths:
    mesh = extract_mesh(nii_path=nii_path, structure_id=structure_id)

    sub, ses = nii_path.split("/")[4:6]
    ply_path = os.path.join(
        MESHES_DIR, f"{hemisphere}_structure_{structure_id}_{sub}_{ses}.ply"
    )
    io.write_trimesh_to_ply(mesh=mesh, ply_path=ply_path)

Found 25 image paths:
 ['/home/data/28andme/sub-01/ses-02/ashs/right_lfseg_corr_usegray_CT_LQ.nii.gz', '/home/data/28andme/sub-01/ses-03/ashs/right_lfseg_corr_usegray_CT_LQ.nii.gz', '/home/data/28andme/sub-01/ses-04/ashs/right_lfseg_corr_usegray_CT_LQ.nii.gz', '/home/data/28andme/sub-01/ses-06/ashs/right_lfseg_corr_usegray_CT_LQ.nii.gz', '/home/data/28andme/sub-01/ses-07/ashs/right_lfseg_corr_usegray_CT_LQ.nii.gz', '/home/data/28andme/sub-01/ses-08/ashs/right_lfseg_corr_usegray_CT_LQ.nii.gz', '/home/data/28andme/sub-01/ses-10/ashs/right_lfseg_corr_usegray_CT_LQ.nii.gz', '/home/data/28andme/sub-01/ses-11/ashs/right_lfseg_corr_usegray_CT_LQ.nii.gz', '/home/data/28andme/sub-01/ses-12/ashs/right_lfseg_corr_usegray_CT_LQ.nii.gz', '/home/data/28andme/sub-01/ses-14/ashs/right_lfseg_corr_usegray_CT_LQ.nii.gz', '/home/data/28andme/sub-01/ses-16/ashs/right_lfseg_corr_usegray_CT_LQ.nii.gz', '/home/data/28andme/sub-01/ses-17/ashs/right_lfseg_corr_usegray_CT_LQ.nii.gz', '/home/data/28andme/sub-01/s